In [4]:
import pandas as pd
import tqdm
import glob
import os

In [5]:
files = glob.glob("../data/wikipedia_fixed/a.parquet")
files = [f for f in files if "all" not in f]

In [6]:
chunk_size = 10_000_000
#chunk_size = 2_750_000

In [7]:
def output_seperated_files(token_length, stride, drop_category):    
    output_dir = f"../data/wikipedia_fixed/sep_a2/token_length{token_length}_stride_sentence{stride}_drop_category{drop_category}"
    os.makedirs(output_dir, exist_ok=True)
    chunk = 1
    df_wiki_sep = []
    for file in files:
        df_wiki = pd.read_parquet(file)
        df_wiki = df_wiki.drop("category", axis=1)
        for i in tqdm.tqdm(range(len(df_wiki))):
            series = df_wiki.iloc[i]
            text = series["text"].replace("\xa0", "")
            if drop_category:
                idx = text.find(" Category:")
                text = text[:idx]

            text_space = text.split(".")
            text = ""
            text_stride = []

            series_ = series.to_dict()
            series_["text"] = ".".join(text_space[:stride*2])
            series_["title_only"] = True
            df_wiki_sep.append(series_)
            
            appended = False
            for j in range(0, len(text_space)):
                series_ = series.to_dict()
                text += text_space[j] + ". "
                text_stride.append(text_space[j])
                appended = False
                if len(text.split(" ")) > token_length:
                    series_["text"] = text
                    series_["title_only"] = False
                    df_wiki_sep.append(series_)
                    text_stride = text_stride[stride:]
                    text = ".".join(text_stride)
                    appended = True
            
            if not appended:
                series_["text"] = text
                series_["title_only"] = False
                df_wiki_sep.append(series_)

            if len(df_wiki_sep) > chunk_size:
                print(f"chunk {chunk}")
                df_wiki_sep = pd.DataFrame(df_wiki_sep)
                df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
                df_wiki_sep = []
                chunk += 1
    df_wiki_sep = pd.DataFrame(df_wiki_sep)
    df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
    

In [8]:
output_seperated_files(token_length=120, stride=4, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [04:48<00:00, 1622.50it/s]


In [9]:
output_seperated_files(token_length=120, stride=6, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [04:45<00:00, 1640.65it/s]


In [10]:
output_seperated_files(token_length=120, stride=8, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [04:41<00:00, 1661.08it/s]


In [11]:
output_seperated_files(token_length=120, stride=10, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [04:36<00:00, 1693.98it/s]


In [12]:
output_seperated_files(token_length=150, stride=10, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [05:20<00:00, 1461.48it/s]


In [13]:
output_seperated_files(token_length=150, stride=12, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [05:38<00:00, 1384.08it/s]


In [14]:
output_seperated_files(token_length=90, stride=4, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [05:24<00:00, 1441.49it/s]


In [15]:
output_seperated_files(token_length=90, stride=6, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [05:07<00:00, 1519.33it/s]


In [16]:
output_seperated_files(token_length=150, stride=6, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467914/467914 [05:30<00:00, 1416.36it/s]


In [42]:
df_a = pd.read_parquet("../data/wikipedia_fixed/a.parquet")

In [25]:
df1 = pd.read_parquet("../data/wikipedia_fixed/sep_a/token_length90_stride_sentence3_drop_categoryTrue/chunk1.parquet")

In [26]:
df2 = pd.read_parquet("../data/wikipedia/sep4/token_length90_stride_sentence3_drop_categoryTrue/a.parquet")

In [55]:
df1["text"].values[12]

' Releasing their debut album in 1994, they are regarded as the pioneers of stoner metal in the United Kingdom, and an important influence upon the scene. Although the band never achieved mainstream success, during their career they received much critical acclaim – they were nominated for the Kerrang! Best Newcomer award and earned top review ratings. Acrimony have maintained a cult following in the British metal scene, their records reportedly selling for vast sums as collectors items AllMusic described their musical style as a "powerful blend of Black Sabbath’s heavy metal riffery, Hawkwind\'s space rock excursions, and Blue Cheer\'s fuzzed-out psychedelic feedback.'

In [39]:
df1.sort_values("id")

,title,text,id,title_only
0,Alexandre Rousselet,Alexandre Rousselet (born 29 January 1977) is ...,10000040,True
1,Alexandre Rousselet,Alexandre Rousselet (born 29 January 1977) is ...,10000040,False
2,Alexandre Rousselet,His best individual finish at the Winter Olym...,10000040,False
3,Alexandre Rousselet,Rousselet's best finish at the FIS Nordic Wor...,10000040,False
4,Alexandre Rousselet,"Rousselet was born in Pontarlier, Doubs. All ...",10000040,False
...,...,...,...,...
6160043,Asylum shopping,It is used mostly in the context of the Europ...,9852952,False
6160042,Asylum shopping,Asylum shopping is a term for the practice by ...,9852952,False
6160040,Asylum shopping,Asylum shopping is a term for the practice by ...,9852952,True
6160049,Asylum shopping,"However, among some asylum seekers, fingerpri...",9852952,False


In [51]:
df_a[df_a["id"] == "10000040"]["text"].values[0].replace("\xa0", "")

'Alexandre Rousselet (born 29 January 1977) is a French cross-country skier who has competed since 1998. His best individual finish at the Winter Olympics was 19th in the 15km event at Turin in 2006. Rousselet\'s best finish at the FIS Nordic World Ski Championships was fifth in the 4 × 10km relay at Sapporo in 2007 while his best individual finish was 25th in the 15km event in 2005. His best individual career finish was fourth in a 15km + 15km double pursuit FIS race in France in 2006 while his best individual World Cup finish was eighth in a 30km event in Italy, also in 2006. Rousselet was born in Pontarlier, Doubs. All results are sourced from the International Ski Federation (FIS). 1 victory – (1 RL) 4 podiums – (4 RL) "ROUSSELET Alexandre". FIS-Ski. International Ski Federation. Retrieved 21 November 2019. Alexandre Rousselet at the International Ski and Snowboard Federation v t e'

In [60]:
df_a.iloc[:20]

,title,text,id,category
12,Alexandre Rousselet,Alexandre Rousselet (born 29 January 1977) is ...,10000040,"[Articles with short description, Short descri..."
30,A Intrusa,A Intrusa is a 1979 Brazilian drama film direc...,10000156,"[CS1 Portuguese-language sources (pt), Article..."
67,Acrimony (band),Acrimony was a Welsh heavy metal band from Swa...,10000383,"[Articles with short description, Short descri..."
73,Atocha station memorial,The Atocha station memorial is a memorial monu...,10000495,"[Articles with short description, Short descri..."
117,Araquistáin,Araquistáin (Basque: Arakistain) is a Spanish ...,10001050,"[Articles containing Basque-language text, Sho..."
176,Alifereti Doviverata,Ro Alifereti Raivalita Doviverata (born 14 Jun...,10001728,"[Articles with short description, Short descri..."
192,Alimqul,"`Alimqul (also spelt Alymkul, Alim quli, Alim ...",10001923,"[Articles with FAST identifiers, Articles with..."
195,Aleksandar Matanović,Aleksandar Matanović (Serbo-Croatian Cyrillic:...,10001965,"[Articles with short description, Short descri..."
232,Argentina–Chile relations,International relations between the Republic o...,10002382,"[CS1: Julian–Gregorian uncertainty, All articl..."
240,Assist (basketball),"In basketball, an assist is attributed to a pl...",1000246,"[Webarchive template wayback links, Articles w..."


In [50]:
df1.sort_values("id").iloc[0]["text"]

"Alexandre Rousselet (born 29 January 1977) is a French cross-country skier who has competed since 1998  His best individual finish at the Winter Olympics was 19th in the 15\xa0km event at Turin in 2006  Rousselet's best finish at the FIS Nordic World Ski Championships was fifth in the 4 × 10\xa0km relay at Sapporo in 2007 while his best individual finish was 25th in the 15\xa0km event in 2005  His best individual career finish was fourth in a 15\xa0km + 15\xa0km double pursuit FIS race in France in 2006 while his best individual World Cup finish was eighth in a 30\xa0km event in Italy, also in 2006  Rousselet was born in Pontarlier, Doubs  All results are sourced from the International Ski Federation (FIS)"

In [48]:
df2.sort_values("id").iloc[0]["text"]

' Rousselet was born in Pontarlier, Doubs. ==Cross-country skiing results== All results are sourced from the International Ski Federation (FIS). ===Olympic Games=== Year Age 15 km Pursuit 30 km 50 km Sprint 4 × 10 km relay Team sprint 2002 25 — 35 47 — — 8 2006 29 18 26 26 — 4 — ===World Championships=== Year Age 15 km Pursuit 30 km 50 km Sprint 4 × 10 km relay Team sprint 2001 24 — 44 — 40 33 — 2003 26 60 — — 47 — 11 2005 28 25 35 26 — 6 — 2007 30 38 44 42 — 5 — 2009 32 27 — — — 9 — ===World Cup=== ====Season standings==== Season Age Discipline standings Ski Tour standings Overall Distance Sprint Tour de Ski World Cup Final 2001 24 126 2002 25 2003 26 106 2004 27 127 88 — 2005 28 77 49 — 2006 29 33 21 — 2007 30 71 41 42 2008 31 120 67 37 42 2009 32 91 62 92 30 — 2010 33 — — — ====Team podiums==== * 1 victory – (1 ) * 4 podiums – (4 ) No Season Date Location Race Level Place Teammates 1 2003–04 7 February 2004 La Clusaz, France 4 × 10 km Relay C/F World Cup 1st Perrillat-Collomb / Vitt

In [61]:
df_gpt = pd.read_parquet("output_gpt3.5_generate/20230922073516.parquet")

In [64]:
df_gpt.head(250).to_csv("only_a_250text_3.csv", index=False)